This file processes all data to have a 'date' and 'hr_beg' column (in UTC to deal with DST issues) and to have one row per hour (ie, to be ready to be combined into a ML-ready dataframe). energy prices and AS prices still could use more DST troubleshooting if there's time, since they skip from 0 to 2 instead of 1 to 3 in hr_beg for march

In [5]:
import json
import csv
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon
from shapely.affinity import scale
import matplotlib.pyplot as plt
import glob
import os
import datetime
import pytz
from pytz import timezone
import pickle

In [16]:
def get_utc(df):
    """Requires dataframe with column 'dt' with datetime"""
    central = timezone('America/Chicago')
    df['Central'] = df['dt'].apply(lambda x: central.localize(x))
    df['UTC'] = df['Central'].apply(lambda x: pytz.utc.normalize(x))
    return df

#  

# 1. AS prices -- DONE
Was badly encoded originally for DST; still has problems in November, but March should be resolved

In [2]:
path = '/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/data_dump_will/'
df_as = pd.read_csv(path+'AS_price_v3.csv')
df_as['dt'] = pd.to_datetime(df_as['Local Datetime (Hour Beginning)'])

In [20]:
#getting timezones to UTC
df_as = get_utc(df_as)

In [49]:
#correcting for march errors, anyway (March should have no 2am, should be 1am)
dst_years = np.arange(2008,2020)
dst_start_days = [9, 8, 14, 13, 11, 10, 9, 8, 13, 12, 11, 10] #march. 
dst_end_days = [2, 1, 7, 6, 4, 3, 2, 1, 6, 5, 4, 3] #nov
start_dates = []
end_dates = []

for i, year in enumerate(dst_years):
    start_dates.append(pd.Timestamp(datetime.datetime(year,3,dst_start_days[i],2,0)))
    end_dates.append(datetime.datetime(year,11,dst_end_days[i],2,0))

for start in start_dates:
    df_as['dt'][df_as['dt']==start] = df_as['dt'][df_as['dt']==start] - datetime.timedelta(hours = 1)
df_as = get_utc(df_as)

In [50]:
#extracting hr_beg and date from UTC datetime
df_as['date'] = df_as['UTC'].dt.date
df_as['hr_beg'] = df_as['UTC'].dt.hour

In [54]:
df_as = df_as[['Market','Price Type','date','hr_beg','Price $/MWh','Volume MWh']]
df_as.columns = ['market','product','date','hr_beg','price','volume']

In [55]:
df_as.drop_duplicates(keep='first', inplace=True)

In [57]:
products = ['Down Regulation', 'Non-Spinning Reserve', 'Responsive Reserve','Up Regulation']
new_products = ['REGDN','NSPIN','RRS','REGUP']
market = 'DAH'

as_output = df_as.loc[:,'date':'hr_beg']

for i, prod in enumerate(products):
    subset = df_as.loc[(df_as['market']==market) & (df_as['product']==prod),['date','hr_beg','price','volume']].rename(columns={'price':'price'+"_"+market+"_"+new_products[i],
                                                                                                                'volume':'vol'+"_"+market+"_"+new_products[i]})
    as_output = as_output.merge(subset, how="outer", on=['date','hr_beg'])

In [59]:
as_output.drop_duplicates(inplace=True)
as_output.reset_index(inplace=True, drop=True)

In [62]:
as_output.to_csv("df_AS_price_vol.csv", index=False) #hr_beg now in utc

#  



# 2. AS Plan -- DONE

In [70]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/DAM AS Plan'
all_files = glob.glob(path + "/*.csv")
df_plan = pd.concat((pd.read_csv(f) for f in all_files))

In [71]:
df_plan.drop_duplicates(subset=['DeliveryDate','HourEnding','AncillaryType','Quantity'], 
                        keep="first", inplace=True)
df_plan.reset_index(inplace=True, drop=True)

In [72]:
#combining date and time to single datetime (and converting to hr_beg to deal with 24:00)
df_plan['hr_end'] = df_plan['HourEnding'].apply(lambda x: int(x[:2]))
df_plan['HourBeginning'] = df_plan['hr_end'] - 1
df_plan.drop(columns=['hr_end'],inplace=True)

In [74]:
df_plan['HourBeginning_str'] = df_plan['HourBeginning'].astype(str)

In [75]:
df_plan['HourBeginning_str'][df_plan['HourBeginning']>=10] = df_plan['HourBeginning'][df_plan['HourBeginning']>=10].astype(str) + ":00"
df_plan['HourBeginning_str'][df_plan['HourBeginning']<10] = "0" + df_plan['HourBeginning'][df_plan['HourBeginning']<10].astype(str) + ":00"

In [76]:
df_plan['dt'] = pd.to_datetime(df_plan['DeliveryDate'] + " " + df_plan['HourBeginning_str'])

In [79]:
df_plan = get_utc(df_plan)

In [81]:
#extracting hr_beg and date from UTC datetime
df_plan['date'] = df_plan['UTC'].dt.date
df_plan['hr_beg'] = df_plan['UTC'].dt.hour

In [85]:
df_plan.drop(columns=['HourEnding','DSTFlag','DeliveryDate','HourBeginning',
                     'HourBeginning_str','dt','Central','UTC'],inplace=True)

In [87]:
products = df_plan['AncillaryType'].unique()
output = df_plan.loc[df_plan['AncillaryType']==products[0],['date','hr_beg','Quantity']]
output.rename(columns={'Quantity':products[0]+"_"+'Quantity'}, inplace=True)

for prod in products[1:]:
    x = df_plan.loc[df_plan['AncillaryType']==prod, ['date','hr_beg','Quantity']]
    output = output.merge(x, how='outer', on=['date','hr_beg'])
    output.rename(columns={'Quantity':prod+"_"+'Quantity'}, inplace=True)

In [90]:
output.to_csv("df_as_plan.csv", index=False)

#   


# 3. AS Bids

In [2]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/Aggregated Ancillary Service Offer Curve'
all_files = glob.glob(path + "/*.csv")
df_bids = pd.concat((pd.read_csv(f) for f in all_files))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [3]:
df_bids['AncillaryType'].unique()

array(['OFFNS', 'ONNS', 'REGDN', 'REGUP', 'RRSGN', 'RRSNC', 'RRSLD'],
      dtype=object)

In [4]:
df_bids.drop_duplicates(subset=['AncillaryType','DeliveryDate','HourEnding','Price','Quantity'],
                   keep='first', inplace=True)
df_bids.reset_index(inplace=True, drop=True)

In [5]:
#combining date and time to single datetime (and converting to hr_beg to deal with 24:00)
df_bids['hr_end'] = df_bids['HourEnding'].apply(lambda x: int(x[:2]))
df_bids['HourBeginning'] = df_bids['hr_end'] - 1
df_bids.drop(columns=['hr_end'],inplace=True)

In [7]:
import pickle
filename = 'df_bids_all.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(df_bids, fp)

In [8]:
df_bids['HourBeginning_str'] = df_bids['HourBeginning'].astype(str)

In [9]:
df_bids['HourBeginning_str'][df_bids['HourBeginning']>=10] = df_bids['HourBeginning'][df_bids['HourBeginning']>=10].astype(str) + ":00"
df_bids['HourBeginning_str'][df_bids['HourBeginning']<10] = "0" + df_bids['HourBeginning'][df_bids['HourBeginning']<10].astype(str) + ":00"

In [10]:
df_bids['dt'] = pd.to_datetime(df_bids['DeliveryDate'] + " " + df_bids['HourBeginning_str'])

In [11]:
unique_dates = df_bids['dt'].unique()

In [13]:
unique_dates = pd.DataFrame({'dt':unique_dates})

In [17]:
unique_dates = get_utc(unique_dates)

In [29]:
unique_utc = unique_dates[['dt','UTC']]

In [30]:
unique_utc.index=unique_utc['dt']

In [31]:
unique_utc.drop(columns=['dt'], inplace=True)

In [33]:
unique_utc = unique_utc.to_dict()

In [49]:
df_bids['UTC'] = df_bids['dt'].map(unique_utc['UTC'])

In [50]:
df_bids['UTC'][0]

Timestamp('2014-01-12 06:00:00+0000', tz='UTC')

In [51]:
#extracting hr_beg and date from UTC datetime
df_bids['date'] = df_bids['UTC'].dt.date
df_bids['hr_beg'] = df_bids['UTC'].dt.hour

In [52]:
df_bids.head()

,AncillaryType,DSTFlag,DeliveryDate,HourEnding,Price,Quantity,HourBeginning,HourBeginning_str,dt,UTC,date,hr_beg
0,OFFNS,N,01/12/2014,01:00,0.01,214.0,0,00:00,2014-01-12,2014-01-12 06:00:00+00:00,2014-01-12,6
1,OFFNS,N,01/12/2014,01:00,50.01,2039.0,0,00:00,2014-01-12,2014-01-12 06:00:00+00:00,2014-01-12,6
2,OFFNS,N,01/12/2014,01:00,60.01,2104.0,0,00:00,2014-01-12,2014-01-12 06:00:00+00:00,2014-01-12,6
3,OFFNS,N,01/12/2014,01:00,10.00,1891.0,0,00:00,2014-01-12,2014-01-12 06:00:00+00:00,2014-01-12,6
4,OFFNS,N,01/12/2014,01:00,5.00,1869.0,0,00:00,2014-01-12,2014-01-12 06:00:00+00:00,2014-01-12,6


In [53]:
df_bids.drop(columns=['HourEnding','HourBeginning','HourBeginning_str','DSTFlag','dt','UTC','DeliveryDate'], inplace=True)

In [54]:
import pickle
filename = 'df_bids_justincase.pickle'
with open(filename, 'wb') as fp:
    pickle.dump(df_bids, fp)

### Grouping bid data
Original version

In [113]:
grouped = df_bids.groupby(['AncillaryType','dt'])

In [114]:
aggregation = {
    'Unweighted Average Price': pd.NamedAgg(column='Price', aggfunc='mean'),
    'Max Price': pd.NamedAgg(column='Price', aggfunc='max'),
    'Min Price': pd.NamedAgg(column='Price', aggfunc='min'),
    'Total Quantity': pd.NamedAgg(column='Quantity', aggfunc='sum'),
    'Number of Bids': pd.NamedAgg(column='Price', aggfunc='size')
}

In [115]:
#want weighted average price
def wavg(group, avg_name, weight_name):
    """ https://pbpython.com/weighted-average.html
    """
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return d.mean()

In [116]:
x = pd.Series(grouped.apply(wavg, "Price", "Quantity"), name="Weighted Avg Price")
grouped_data = pd.concat([grouped.agg(**aggregation), x], axis=1)

In [119]:
products = df_bids['AncillaryType'].unique()
output = grouped_data.loc[(products[0]),:]
output.columns = [products[0] + "_" + str(col) for col in output.columns]

for prod in products[1:]:
    x = grouped_data.loc[(prod),:]
    x.columns = [prod + "_" + str(col) for col in x.columns]
    output = pd.concat([output, x], axis=1)

In [121]:
output.reset_index(level=0, inplace=True)

In [126]:
output = get_utc(output)

In [128]:
#extracting hr_beg and date from UTC datetime
output['date'] = output['UTC'].dt.date
output['hr_beg'] = output['UTC'].dt.hour

In [129]:
output.head(1)

,dt,OFFNS_Unweighted Average Price,OFFNS_Max Price,OFFNS_Min Price,OFFNS_Total Quantity,OFFNS_Number of Bids,OFFNS_Weighted Avg Price,ONNS_Unweighted Average Price,ONNS_Max Price,ONNS_Min Price,ONNS_Total Quantity,ONNS_Number of Bids,ONNS_Weighted Avg Price,REGDN_Unweighted Average Price,REGDN_Max Price,REGDN_Min Price,REGDN_Total Quantity,REGDN_Number of Bids,REGDN_Weighted Avg Price,REGUP_Unweighted Average Price,REGUP_Max Price,REGUP_Min Price,REGUP_Total Quantity,REGUP_Number of Bids,REGUP_Weighted Avg Price,RRSGN_Unweighted Average Price,RRSGN_Max Price,RRSGN_Min Price,RRSGN_Total Quantity,RRSGN_Number of Bids,RRSGN_Weighted Avg Price,RRSNC_Unweighted Average Price,RRSNC_Max Price,RRSNC_Min Price,RRSNC_Total Quantity,RRSNC_Number of Bids,RRSNC_Weighted Avg Price,RRSLD_Unweighted Average Price,RRSLD_Max Price,RRSLD_Min Price,RRSLD_Total Quantity,RRSLD_Number of Bids,RRSLD_Weighted Avg Price,Central,UTC,date,hr_beg
0,2010-10-12,7.505714,20.0,0.01,1396.0,7,9.063517,11.58,18.51,2.0,13277.0,7,13.514449,13.065,60.0,3.0,12273.0,12,14.872349,13.023077,50.0,0.99,11205.0,13,16.162925,10.39,30.0,0.01,23621.0,13,15.215243,0.0,0.0,0.0,335.9,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2010-10-12 00:00:00-05:00,2010-10-12 05:00:00+00:00,2010-10-12,5


In [130]:
output.drop(columns=['dt','Central','UTC'], inplace=True)

In [131]:
output.to_csv("df_as_bid_aggregated_data.csv", index=False)

### Grouping bid data -- new version
Reg down

In [102]:
filename = 'df_bids_justincase.pickle'
with open(filename, 'rb') as fp:
    df_bids = pickle.load(fp)

In [103]:
df_bids = df_bids[df_bids['AncillaryType']=='REGDN']
df_bids.drop(columns=['AncillaryType'], inplace=True)

In [106]:
df_bids.sort_values(by=['date','hr_beg','Price'], inplace=True)
df_bids.reset_index(inplace=True, drop=True)

df_bids['year'] = pd.to_datetime(df_bids['date']).dt.year
df_bids = df_bids[df_bids['year']>2013]
df_bids.reset_index(inplace=True, drop=True)
df_bids.drop(columns=['year'], inplace=True)

In [11]:
def price_at_percentile(group, price_name, quant_name, percentile):
    """ https://pbpython.com/weighted-average.html
    """
    p = group[price_name]
    q = group[quant_name]
    return p[q.where (q > max(q)*percentile).first_valid_index()]

In [110]:
x90 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.9), name="90th Pctl Bid"))

In [113]:
x80 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.8), name="80th Pctl Bid"))

In [114]:
x70 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.7), name="70th Pctl Bid"))

In [118]:
x60 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.6), name="60th Pctl Bid"))
x50 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.5), name="50th Pctl Bid"))
x30 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.3), name="30th Pctl Bid"))

In [125]:
x90.reset_index(inplace=True)
x80.reset_index(inplace=True)
x70.reset_index(inplace=True)
x60.reset_index(inplace=True)
x50.reset_index(inplace=True)
x30.reset_index(inplace=True)

In [128]:
output = x90.merge(x80, how='left', on=['date','hr_beg'])
output = output.merge(x70, how='left', on=['date','hr_beg'])
output = output.merge(x60, how='left', on=['date','hr_beg'])
output = output.merge(x50, how='left', on=['date','hr_beg'])
output = output.merge(x30, how='left', on=['date','hr_beg'])

In [129]:
output.head()

,date,hr_beg,90th Pctl Bid,80th Pctl Bid,70th Pctl Bid,60th Pctl Bid,50th Pctl Bid,30th Pctl Bid
0,2014-01-01,0,25.00,11.00,7.13,5.65,3.93,2.08
1,2014-01-01,1,18.55,12.63,8.11,6.48,5.55,2.11
2,2014-01-01,2,19.04,14.11,8.11,6.36,5.19,2.11
3,2014-01-01,3,20.24,14.11,10.00,8.11,5.55,2.11
4,2014-01-01,4,21.68,14.11,11.00,8.11,6.91,2.11


In [130]:
output.to_csv("as_bids_REGDOWN.csv",index=False)

# Reg up
Reg up, which is correlated with reg down prices

In [6]:
filename = 'df_bids_justincase.pickle'
with open(filename, 'rb') as fp:
    df_bids = pickle.load(fp)

In [7]:
df_bids = df_bids[df_bids['AncillaryType']=='REGUP']
df_bids.drop(columns=['AncillaryType'], inplace=True)

In [9]:
df_bids.sort_values(by=['date','hr_beg','Price'], inplace=True)
df_bids.reset_index(inplace=True, drop=True)

df_bids['year'] = pd.to_datetime(df_bids['date']).dt.year
df_bids = df_bids[df_bids['year']>2013]
df_bids.reset_index(inplace=True, drop=True)
df_bids.drop(columns=['year'], inplace=True)

In [13]:
x90 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.9), name="90th Pctl Bid"))

In [14]:
x80 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.8), name="80th Pctl Bid"))

In [15]:
x70 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.7), name="70th Pctl Bid"))
x60 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.6), name="60th Pctl Bid"))
x50 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.5), name="50th Pctl Bid"))
x30 = pd.DataFrame(pd.Series(df_bids.groupby(['date','hr_beg']).apply(price_at_percentile, "Price", "Quantity",.3), name="30th Pctl Bid"))

In [23]:
for d in [x90, x80, x70, x60, x50, x30]:
    d.reset_index(inplace=True)

In [25]:
for d in [x90, x80, x70, x60, x50, x30]:
    d.rename(columns = {d.columns[2]:d.columns[2]+"_REGUP"}, inplace=True)

In [27]:
output = x90.merge(x80, how='left', on=['date','hr_beg'])
output = output.merge(x70, how='left', on=['date','hr_beg'])
output = output.merge(x60, how='left', on=['date','hr_beg'])
output = output.merge(x50, how='left', on=['date','hr_beg'])
output = output.merge(x30, how='left', on=['date','hr_beg'])

In [29]:
output.to_csv("as_bids_REGUP.csv",index=False)

#  

# 4. Energy prices -- DONE
Had DST issues

In [91]:
path = '/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/data_dump_will/'
df_energy = pd.read_csv(path+'energy_price.csv')

In [92]:
#converting to UTC
df_energy['dt'] = pd.to_datetime(df_energy['Local Datetime (Hour Ending)'])
df_energy = get_utc(df_energy)

In [93]:
#converting UTC hour ending to UTC hour beginning, and extracting date and hr_beg from there
df_energy['UTC_hr_beg'] = df_energy['UTC'] - datetime.timedelta(hours = 1)
df_energy['date'] = df_energy['UTC_hr_beg'].dt.date
df_energy['hr_beg'] = df_energy['UTC_hr_beg'].dt.hour

In [95]:
#subsetting to columns of interest
df_energy = df_energy[['Price Node Name','Price Type','Market','date','hr_beg','Price $/MWh']].reset_index(drop=True)
df_energy.columns = ['node','price_type','market','date','hr_beg','price']

In [96]:
#reshaping data
nodes = ['HB_BUSAVG', 'HB_HOUSTON', 'HB_HUBAVG', 'HB_NORTH', 'HB_SOUTH','HB_WEST']
newnodes = ['busavg','houston','hubavg','N','S','W']
markets = ['DAH', 'RT15AVG']
newmarkets = ['DAH','RT15']

energy_output = df_energy.loc[:,'date':'hr_beg']

for i, market in enumerate(markets):
    for j, node in enumerate(nodes):
        subset = df_energy.loc[(df_energy['market']==market) & (df_energy['node']==node),['date','hr_beg','price']].rename(columns={'price':'price'+"_"+newmarkets[i]+"_"+newnodes[j],
                                                                                                                    })
        energy_output = energy_output.merge(subset, on=['date','hr_beg'], how="outer")

In [97]:
energy_output.drop_duplicates(inplace=True) #why so many dupes?
energy_output.reset_index(inplace=True, drop=True)

In [99]:
energy_output.to_csv("df_energy_price.csv", index=False)

#  


# 5. Generation -- DONE

In [100]:
path = '/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/data_dump_will/'
df_gen = pd.read_csv(path+'ERCOT_hourly_by_BA_v5.csv')

In [101]:
df_gen['dt'] = pd.to_datetime(df_gen['datetime']) #this seems to be central time
df_gen = get_utc(df_gen)

In [102]:
df_gen['date'] = df_gen['UTC'].dt.date
df_gen['hr_beg'] = df_gen['UTC'].dt.time

In [103]:
df_gen.drop(columns=['local_time_cems','utc','datetime','UTC','Central','dt'], inplace=True)

In [106]:
df_gen.drop_duplicates(inplace=True)
df_gen.reset_index(inplace=True, drop=True)

In [107]:
df_gen.to_csv('df_generation.csv', index=False)

#  


# 6.Weather -- DONE

In [3]:
#loading all data and concatenating
path = r'/Users/margaretmccall/Documents/2020 Spring/CE 295/0 - Final Project/Data--ERCOT/Weather_Assumptions'
all_files = glob.glob(path + "/*.csv")
df_weather = pd.concat((pd.read_csv(f) for f in all_files))

In [4]:
df_weather.drop_duplicates(subset=['DeliveryDate','HourEnding'], keep="first", inplace=True)
df_weather.sort_values(by=['DeliveryDate','HourEnding'], inplace=True)
df_weather.reset_index(inplace=True, drop=True)

In [14]:
#combining date and time to single datetime
df_weather['dt'] = pd.to_datetime(df_weather['DeliveryDate'] + " " + df_weather['HourEnding'])

In [19]:
#getting timezones to UTC
df_weather = get_utc(df_weather)

In [22]:
#converting UTC hour ending to UTC hour beginning, and extracting date and hr_beg from there
df_weather['UTC_hr_beg'] = df_weather['UTC'] - datetime.timedelta(hours = 1)

In [24]:
df_weather['date'] = df_weather['UTC_hr_beg'].dt.date
df_weather['hr_beg'] = df_weather['UTC_hr_beg'].dt.hour

In [29]:
df_weather.drop(columns=['DeliveryDate','HourEnding','DSTFlag','dt','Central','UTC','UTC_hr_beg'], inplace=True)

In [31]:
#saving. hr_beg is now in UTC
df_weather.to_csv('weather_forecast_ercot.csv', index=False)